In [1]:
!pip install pip3-autoremove
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu128
!pip install unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

Looking in indexes: https://download.pytorch.org/whl/cu128


In [2]:
!pip install --upgrade torchvision --index-url https://download.pytorch.org/whl/cu128
!pip install --upgrade unsloth

Looking in indexes: https://download.pytorch.org/whl/cu128


In [3]:
!pip install git+https://github.com/huggingface/peft.git@main

  Cloning https://github.com/huggingface/peft.git (to revision main) to /tmp/pip-req-build-6yrfz2v1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-6yrfz2v1
  Resolved https://github.com/huggingface/peft.git to commit 261366de2e40cde64b702d6b9c527081ad850549
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
!pip install evaluate --quiet

In [5]:
# import os
# os._exit(00)

In [6]:
import torch, torchvision
print(torch.__version__)
print(torchvision.__version__)

2.9.1+cu128
0.24.1+cu128


In [7]:
import unsloth
print(unsloth.__version__)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

🦥 Unsloth Zoo will now patch everything to make training faster!
2025.12.10


In [8]:
import peft
print(peft.__version__) 

0.18.1.dev0


In [9]:
import bitsandbytes as bnb
print(bnb.__version__)

0.49.0


In [10]:
import unsloth
from unsloth import FastLanguageModel
from datasets import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments
from trl import SFTTrainer
import torch
import evaluate
import csv
from datetime import datetime

In [11]:
class DatasetProcessor:
    def __init__(self, csv_path):
        self.df = pd.read_csv(csv_path)
    
    def preprocess(self):
        self.df = self.df.dropna(subset=["Topics", "Question-Title", "Questions", "Answers"])
        self.df["input_text"] = (
            "বিষয়: " + self.df["Topics"].astype(str) + "\n"
            "শিরোনাম: " + self.df["Question-Title"].astype(str) + "\n\n"
            "প্রশ্ন:\n" + self.df["Questions"].astype(str)
        )
        self.df["target_text"] = self.df["Answers"].astype(str)
        
        train_df, val_df = train_test_split(self.df, test_size=0.01, random_state=42)
        train_dataset = Dataset.from_pandas(train_df[["input_text", "target_text"]].reset_index(drop=True))
        val_dataset   = Dataset.from_pandas(val_df[["input_text", "target_text"]].reset_index(drop=True))
        return train_dataset, val_dataset

In [12]:
PROMPT_TEMPLATE = """### Instruction:
আপনি একজন অভিজ্ঞ কাউন্সেলর এবং সহানুভূতিশীল কথোপকথনকারী।
আপনার কাজ হলো বাংলায় এমন উত্তর দেওয়া, যাতে বোঝাপড়া, সহমর্মিতা এবং মানসিক সমর্থন প্রকাশ পায়।

### Input:
{input}

### Response:
{output}"""

def format_prompts(examples):
    texts = []
    for inp, out in zip(examples["input_text"], examples["target_text"]):
        text = PROMPT_TEMPLATE.format(input=inp, output=out)
        texts.append(text)
    return {"text": texts}

In [13]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24951 sha256=0374a2279d4a39eceef3344315259aa339cb89ffdf8e0fc839534beb709c0142
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [14]:
class PEFTStrategy:
    def apply(self, model):
        raise NotImplementedError

class LoRAStrategy(PEFTStrategy):
    def __init__(self, lora_config):
        self.lora_config = lora_config
    
    def apply(self, model):
        return FastLanguageModel.get_peft_model(model, **self.lora_config)

class UnslothStrategy(PEFTStrategy):
    def apply(self, model):
        return FastLanguageModel.get_peft_model(model, use_gradient_checkpointing="unsloth")

In [15]:
class LLAMAFineTuner:
    def __init__(self, model_name, strategy: PEFTStrategy, tokenizer_token=None):
        self.model, self.tokenizer = FastLanguageModel.from_pretrained(
            model_name,
            dtype=None,
            load_in_4bit=True,
            token=tokenizer_token,
            device_map="auto",
        )
        self.model = strategy.apply(self.model)
    
    def train(self, train_dataset, val_dataset, output_dir="Llama3-8B", max_steps=10):
        train_dataset = train_dataset.map(format_prompts, batched=True)
        val_dataset   = val_dataset.map(format_prompts, batched=True)
        
        train_dataset = train_dataset.remove_columns(['input_text', 'target_text'])
        val_dataset   = val_dataset.remove_columns(['input_text', 'target_text'])
        
        training_args = TrainingArguments(
            per_device_train_batch_size=1,
            gradient_accumulation_steps=4,
            max_steps=max_steps,
            learning_rate=2e-4,
            fp16=True,
            bf16=False,
            logging_steps=1,
            eval_strategy="steps",
            eval_steps=1,
            save_steps=10,
            save_strategy="steps",
            output_dir=output_dir,
            report_to="none",
            seed=3407,
            load_best_model_at_end=True,
            metric_for_best_model="eval_loss",
        )
        
        self.trainer = SFTTrainer(
            model=self.model,
            tokenizer=self.tokenizer,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            args=training_args
        )
        self.trainer.train()
        return self.trainer

In [17]:
if __name__ == "__main__":
    csv_path = "/kaggle/input/bangla/BengaliEmpatheticConversationsCorpus .csv"
    processor = DatasetProcessor(csv_path)
    train_dataset, val_dataset = processor.preprocess()
    
    lora_config = {
        "r":4,
        "target_modules":["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
        "lora_alpha":4,
        "lora_dropout":0,
        "bias":"none",
        "use_gradient_checkpointing":"unsloth",
        "random_state":3407,
        "use_rslora":False,
        "loftq_config":None,
    }
    strategy = LoRAStrategy(lora_config)
    tuner = LLAMAFineTuner("unsloth/Meta-Llama-3.1-8B-Instruct", strategy)
    trainer = tuner.train(train_dataset, val_dataset, max_steps=100)

==((====))==  Unsloth 2025.12.10: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

...

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/377 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 2
   \\   /|    Num examples = 37,233 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 10,485,760 of 8,040,747,008 (0.13% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
1,0.959400,0.911158
2,0.869000,0.846916
3,0.807400,0.784941
4,0.804600,0.723975
5,0.772100,0.664747
6,0.725400,0.616953
7,0.580900,0.575551
8,0.570900,0.544800
9,0.621100,0.522653
10,0.541800,0.502602


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


In [18]:
from tqdm import tqdm
import torch
import evaluate
import math

def compute_perplexity(model, tokenizer, dataset, max_length=512):
    model.eval()
    total_loss = 0
    total_tokens = 0

    for ex in tqdm(dataset, desc="Computing Perplexity"):
        text = ex["target_text"]
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_length)
        input_ids = inputs.input_ids.to(model.device)
        
        with torch.no_grad():
            outputs = model(input_ids, labels=input_ids)
            loss = outputs.loss
            total_loss += loss.item() * input_ids.size(1)
            total_tokens += input_ids.size(1)
    
    avg_loss = total_loss / total_tokens
    ppl = math.exp(avg_loss)
    return ppl

eval_dataset = val_dataset.select(range(20))

ppl = compute_perplexity(tuner.model, tuner.tokenizer, eval_dataset)
print("Perplexity:", ppl)


Computing Perplexity: 100%|██████████| 20/20 [00:08<00:00,  2.30it/s]

Perplexity: 2.7268978129904693


In [19]:
from tqdm import tqdm
import torch

tuner.model.eval()

eval_dataset = val_dataset.select(range(5))  

def extract_response(text):
    if "### Response:\n" in text:
        return text.split("### Response:\n")[-1].strip()
    return text.strip()

In [20]:
from tqdm import tqdm
import torch

tuner.model.eval()
eval_dataset = val_dataset.select(range(10))
def extract_response(text):
    if "### Response:\n" in text:
        return text.split("### Response:\n")[-1].strip()
    return text.strip()
examples = []
for ex in tqdm(eval_dataset, desc="Generating examples"):
    input_text = ex["input_text"]
    inputs = tuner.tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(tuner.model.device)
    
    with torch.no_grad():
        output_ids = tuner.model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=True,
            top_p=0.7,
            temperature=0.6
        )
    
    decoded = tuner.tokenizer.decode(output_ids[0], skip_special_tokens=True)
    prediction = extract_response(decoded)
    target_text = ex["target_text"]
    if "### Response:\n" in target_text:
        reference = target_text.split("### Response:\n")[-1].strip()
    else:
        reference = target_text.strip()
    examples.append((input_text, prediction, reference))


Generating examples: 100%|██████████| 10/10 [06:40<00:00, 40.04s/it]


In [21]:
for idx, (inp, pred, ref) in enumerate(examples):
    print(f"---Example---")
    print("Input     :", inp)
    print("Prediction:", pred)
    print("Reference:", ref)
    print()

---Example---
Input     : বিষয়: বিব্রত
শিরোনাম: অসারতা

প্রশ্ন:
এটা আমার অহংকার জন্য আরো বেদনাদায়ক ছিল!
Prediction: বিষয়: বিব্রত
শিরোনাম: অসারতা

প্রশ্ন:
এটা আমার অহংকার জন্য আরো বেদনাদায়ক ছিল! আমি সব সময় বিশ্বাস করতে পারি না যে আমি একটি অসার বাবা হতে পারি।

উত্তর:
আমি একটি অসার বাবা হতে পারি না এবং আমি এটা সম্পর্কে কিছু করতে পারি না। আমি আমার পুত্রকে আমার সাথে কি করতে হবে তা জানি না। আমি আমার পুত্রকে কি করতে হবে তা জানি না। আমি আমার পুত্রকে কি করতে হবে তা জানি না। আমি আমার পুত্রকে কি করতে হবে তা জানি না। আমি আমার পুত্রকে কি করতে হবে তা জানি না। আমি আমার পুত্রকে কি করতে হবে তা জানি না। আমি আমার পুত্রকে কি করতে হবে তা জানি ন
Reference: আমি জানি তুমি কেমন অনুভব করছো. অন্তত ইউটিউবে নয়

---Example---
Input     : বিষয়: প্রস্তুত
শিরোনাম: সপ্তাহ

প্রশ্ন:
আমরা এক সপ্তাহের জন্য ছুটিতে যাচ্ছিলাম। আমার পাঁচটি বাচ্চা আছে তাই আমি তাদের নিজের ব্যাগে প্রতিদিনের কাপড় রাখি। খুব সংগঠিত ছিলাম।
Prediction: বিষয়: প্রস্তুত
শিরোনাম: সপ্তাহ

প্রশ্ন:
আমরা এক সপ্তাহের জন্য ছুটিতে যাচ্ছিলাম। আমার পাঁচটি বাচ

In [22]:
test_prompt = "আমার পোষা কচ্ছপ মারা গেছে।"
inputs = tuner.tokenizer(test_prompt, return_tensors="pt").to(tuner.model.device)

with torch.no_grad():
    generated_ids = tuner.model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=True,
        top_p=0.4,
        temperature=0.3,
    )

generated_text = tuner.tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("Model Output:\n", generated_text)

Model Output:
 আমার পোষা কচ্ছপ মারা গেছে। আমি এটা কি করতে পারি?
আমি একটি কচ্ছপ পোষা করি এবং আমার কচ্ছপ মারা গেছে। আমি এটা কি করতে পারি?
আমি একটি কচ্ছপ পোষা করি এবং আমার কচ্ছপ মারা গেছে। আমি এটা কি করতে পারি?
আমি একটি কচ্ছপ পোষা করি এবং আমার কচ্ছপ মারা গেছে। আমি এটা কি করতে পারি?
আমি একটি কচ্ছপ পোষা করি এবং আমার কচ্ছপ মারা গেছে। আমি এটা কি করতে পারি?
আমি একটি কচ্ছপ পোষা করি এবং আমার কচ্ছপ মারা গেছে। আমি এটা কি করতে পারি?
আমি একটি কচ্ছপ পোষা করি এবং আমার কচ


In [23]:
test_prompt = "আমি সত্যিই আমার পদার্থবিদ্যার ডিগ্রী বের হওয়ার জন্য অপেক্ষা করছিলাম এবং আমি ক্লাসে ভালো করেছি এটা দেখে খুশি হয়েছিলাম।"
inputs = tuner.tokenizer(test_prompt, return_tensors="pt").to(tuner.model.device)

with torch.no_grad():
    generated_ids = tuner.model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=True,
        top_p=0.7,
        temperature=0.6,
    )

generated_text = tuner.tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("Model Output:\n", generated_text)

Model Output:
 আমি সত্যিই আমার পদার্থবিদ্যার ডিগ্রী বের হওয়ার জন্য অপেক্ষা করছিলাম এবং আমি ক্লাসে ভালো করেছি এটা দেখে খুশি হয়েছিলাম। আমি সত্যিই আমার পদার্থবিদ্যার ডিগ্রী বের হওয়ার জন্য অপেক্ষা করছিলাম এবং আমি ক্লাসে ভালো করেছি এটা দেখে খুশি হয়েছিলাম।
আমি সত্যিই আমার পদার্থবিদ্যার ডিগ্রী বের হওয়ার জন্য অপেক্ষা করছিলাম এবং আমি ক্লাসে ভালো করেছি এটা দেখে খুশি হয়েছিলাম।
আমি সত্যিই আমার পদার্থবিদ্যার ডিগ্রী বের হওয়ার জন্য অপেক্ষা করছিলাম এবং আমি ক্লাসে ভালো করেছি এটা দেখে খুশি হয়েছিলাম।
আমি সত্যিই আমার পদার্থবিদ্যার ডিগ্রী বের হওয়ার জন্য অপেক্ষা করছিলাম এবং আমি ক্লাসে ভালো করেছি এটা দেখে খুশি হয়েছিলাম।
আমি সত্যিই আমার পদার্থবিদ্যার ডিগ্রী বের হওয়ার জন্য অপেক্ষা করছিলাম এবং আমি ক্লাসে ভালো করেছি এটা দেখে খুশি হয়েছিলাম।
আমি সত্যিই আমার পদার্থবিদ্যার ডিগ্রী বের হওয়ার জন্য অপেক্ষা করছিলাম এবং আমি ক্লাসে ভালো করেছি এটা দেখে খুশি হয়েছিলাম।
আমি সত্যিই আমার পদার্থবিদ্যার ডিগ্রী বের হওয়ার জন্য অপেক্ষা করছিলাম এবং আমি ক্লাসে ভালো করেছি এটা দেখে খুশি হয়েছিলাম।
আমি সত্যিই আ�


In [25]:
test_prompt =  "কাউন্সেলিং কি সত্যিই এমন কিছু করে যা মানুষকে সাহায্য করতে পারে? আমি কিভাবে এটি ব্যবহার করতে পারি?"
inputs = tuner.tokenizer(test_prompt, return_tensors="pt").to(tuner.model.device)

with torch.no_grad():
    generated_ids = tuner.model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=True,
        top_p=0.4,
        temperature=0.5,
    )

generated_text = tuner.tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("Model Output:\n", generated_text)

Model Output:
 কাউন্সেলিং কি সত্যিই এমন কিছু করে যা মানুষকে সাহায্য করতে পারে? আমি কিভাবে এটি ব্যবহার করতে পারি? কাউন্সেলিং কি সত্যিই এমন কিছু করে যা মানুষকে সাহায্য করতে পারে? আমি কিভাবে এটি ব্যবহার করতে পারি?
কাউন্সেলিং একটি সম্পর্ক যা একজন কাউন্সেলর এবং তাদের ক্লায়েন্টের মধ্যে কাজ করে যাতে ক্লায়েন্ট তাদের জীবনে সমস্যা এবং চ্যালেঞ্জগুলির সাথে নেভিগেট করতে পারে। কাউন্সেলিং একটি সম্পর্ক যা একজন কাউন্সেলর এবং তাদের ক্লায়েন্টের মধ্যে কাজ করে যাতে ক্লায়েন্ট তাদের জীবনে সমস্যা এবং চ্যালেঞ্জগুলির সাথে নেভিগেট করতে পারে।
কাউন্সেলিং একটি সম্পর্ক যা একজন কাউন্সেলর এবং তাদের ক্লায়েন্টের মধ্যে কাজ করে যাতে ক্লায়েন্ট তাদের জীবনে সমস্যা এবং চ্যালেঞ্জগুলির সাথে নেভিগেট করতে পারে।
কাউন্সেলিং একটি সম্পর্ক যা একজন কাউন্সেলর এবং তাদের ক্লায়েন্টের মধ্যে কাজ করে যাতে ক্লায়েন্ট তাদের জীবনে সমস্যা এবং চ্যালেঞ্জগুলির সাথে নেভিগেট করতে পারে।
কাউন্সেলিং একটি সম্পর্ক যা একজন কাউন্সেলর এবং তাদের ক্লায়েন্টের মধ্যে কাজ করে যাতে ক্লায়েন্ট তাদের জীবনে সমস্যা এবং চ্যালেঞ্জগুলির সাথে নেভিগেট করতে পারে।
কা�
